In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import urllib
import time
from selenium.webdriver.chrome.options import Options
import selenium
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from lxml import html
import re
from datetime import datetime, timedelta, date
from IPython.display import clear_output
import pickle
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count
import logging
import io
import traceback
import winsound
import warnings
warnings.filterwarnings("ignore")
from pqdm.threads import pqdm

In [2]:
def yuksekoran(df):
    bests = df.copy()
    bests.columns = df.columns.str.lower()
    bests = bests.drop(['aust_prob','hust_prob','uust_prob','bust_prob','hsust_prob','asust_prob','x1_prob','x2_prob','kg_prob','hbust_prob','abust_prob','ms1_prob','ms2_prob','ms0_prob','ust_prob','evh1_prob'],axis=1)
    bests['max_prob'] = bests.iloc[:,671:].max(axis=1)
    bests['max_col'] = bests.iloc[:,671:].idxmax(axis=1)
    values = []
    for i in bests.index:
        tahmin = bests['max_col'].str.split('_',expand=True)[0][i]
        if tahmin == 'ms1' or tahmin == 'ms2' or tahmin == 'ms0':
            tahmin = tahmin[:-1]
        values.append(bests[tahmin].loc[i])
    bests['max_vals'] = values
    deneme = bests[['home team','away team','max_prob','max_col','max_vals']]
    kupon = deneme.sort_values('max_prob',ascending=False)
    return kupon

def bankokupon(df):
    bests = df.copy()
    bests.columns = df.columns.str.lower()
    bests = bests.drop(['aust_prob','hust_prob','asust_prob','evh1_prob','hbust_prob'],axis=1)
    bests['max_prob'] = bests.iloc[:,671:].max(axis=1)
    bests['max_col'] = bests.iloc[:,671:].idxmax(axis=1)
    values = []
    for i in bests.index:
        tahmin = bests['max_col'].str.split('_',expand=True)[0][i]
        if tahmin == 'ms1' or tahmin == 'ms2' or tahmin == 'ms0':
            tahmin = tahmin[:-1]
        values.append(bests[tahmin].loc[i])
    bests['max_vals'] = values
    deneme = bests[['home team','away team','max_prob','max_col','max_vals']]
    kupon = deneme.sort_values('max_prob',ascending=False)
    return kupon  

def birazbanko(df):
    bests = df.copy()
    bests.columns = df.columns.str.lower()
    bests = bests.drop(['aust_prob','hust_prob','uust_prob','evh1_prob','bust_prob','hsust_prob','asust_prob','x1_prob','x2_prob','kg_prob','hbust_prob','bustx1_prob','bustx2_prob','uustx1_prob','uustx2_prob'],axis=1)
    bests['max_prob'] = bests.iloc[:,671:].max(axis=1)
    bests['max_col'] = bests.iloc[:,671:].idxmax(axis=1)
    values = []
    for i in bests.index:
        tahmin = bests['max_col'].str.split('_',expand=True)[0][i]
        if tahmin == 'ms1' or tahmin == 'ms2' or tahmin == 'ms0':
            tahmin = tahmin[:-1]
        values.append(bests[tahmin].loc[i])
    bests['max_vals'] = values
    deneme = bests[['home team','away team','max_prob','max_col','max_vals']]
    kupon = deneme.sort_values('max_prob',ascending=False)
    return kupon

def idealoran(df):
    bests = df.copy()
    bests.columns = df.columns.str.lower()
    bests = bests.drop(['aust_prob','hust_prob','uust_prob','evh1_prob','bust_prob','hsust_prob','asust_prob','x1_prob','x2_prob','kg_prob','hbust_prob','abust_prob','bustx1_prob','bustx2_prob','uustx1_prob','uustx2_prob'],axis=1)
    bests['max_prob'] = bests.iloc[:,671:].max(axis=1)
    bests['max_col'] = bests.iloc[:,671:].idxmax(axis=1)
    values = []
    for i in bests.index:
        tahmin = bests['max_col'].str.split('_',expand=True)[0][i]
        if tahmin == 'ms1' or tahmin == 'ms2' or tahmin == 'ms0':
            tahmin = tahmin[:-1]
        values.append(bests[tahmin].loc[i])
    bests['max_vals'] = values
    deneme = bests[['home team','away team','max_prob','max_col','max_vals']]
    kupon = deneme.sort_values('max_prob',ascending=False)
    return kupon

def ideal(df):
    bests = df.copy()
    bests.columns = df.columns.str.lower()
    bests = bests.drop(['aust_prob','hust_prob','evh1_prob','uust_prob','bust_prob','hsust_prob','asust_prob','x1_prob','x2_prob','kg_prob','hbust_prob','abust_prob'],axis=1)
    bests['max_prob'] = bests.iloc[:,671:].max(axis=1)
    bests['max_col'] = bests.iloc[:,671:].idxmax(axis=1)
    values = []
    for i in bests.index:
        tahmin = bests['max_col'].str.split('_',expand=True)[0][i]
        if tahmin == 'ms1' or tahmin == 'ms2' or tahmin == 'ms0':
            tahmin = tahmin[:-1]
        values.append(bests[tahmin].loc[i])
    bests['max_vals'] = values
    deneme = bests[['home team','away team','max_prob','max_col','max_vals']]
    kupon = deneme.sort_values('max_prob',ascending=False)
    return kupon

def tables(url,s):
    parameter = f"&tab=form_teams&cmp=0&lm={s}"
    url = url+parameter
    r= requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    stats =pd.read_html(url,flavor='lxml')
    t1data = stats[2].T.reset_index(drop=True).iloc[0].append(stats[2].T.reset_index(drop=True).iloc[2]).reset_index(drop=True)
    t1col = ('Home ' + stats[2].T.reset_index(drop=True).iloc[1]).append('Away ' + stats[2].T.reset_index(drop=True).iloc[1])
    t2data = stats[3].T.reset_index(drop=True).iloc[0].append(stats[3].T.reset_index(drop=True).iloc[2]).reset_index(drop=True)
    t2col = ('Home ' + stats[3].T.reset_index(drop=True).iloc[1]).append('Away ' + stats[3].T.reset_index(drop=True).iloc[1])
    col = t1col.append(t2col)
    datas = t1data.append(t2data).reset_index(drop=True)
    df = pd.DataFrame(columns=col,data = np.array([datas])).add_prefix(f'{s}_')
    return df

def yarat(url):
    try:
        parameter = "&tab=form_teams&cmp=0&lm=5"
        url = url+parameter
        r= requests.get(url)
        soup = BeautifulSoup(r.text,"html.parser")
        stats =pd.read_html(url,flavor='lxml')
        p = re.compile(r'(\d{2}.\d{2}.\d{4}) (\d{2}:\d{2})')
        try:
            evname = soup.find_all('div',{"class":"live_game_ht"})[0].text.replace('\n','')
        except:
            evname = np.nan
        try:
            evid = soup.find_all('div',{"class":"live_game_ht"})[0].find('a')['href'].split('/')[-2]
        except:
            evid = np.nan
        try:
            depid = soup.find_all('div',{"class":"live_game_at"})[0].find('a')['href'].split('/')[-2]
        except:
            depid = np.nan
        try:
            depname = soup.find_all('div',{"class":"live_game_at"})[0].text.replace('\n','')
        except:
            depname = np.nan
        try:
            evgoal = int(soup.find_all('div',{"class":"live_game_goal"})[0].text.replace('\n',''))
        except:
            evgoal = np.nan
        try:    
            depgoal = int(soup.find_all('div',{"class":"live_game_goal"})[1].text.replace('\n',''))
        except:
            depgoal = np.nan
        try:
            league = soup.find_all('div',{"class":"breadcrumb"})[0].find_all('a',href=True)[2]['href'].split('/')[-2]
        except:
            league = np.nan
        try:
            text = soup.find_all('div',{"class":"block_header bkcenter"})[0].text.strip('\n')
        except:
            text = np.nan

        data = {'Home Team':evname,'Home ID':evid,'Away Team':depname,'Away ID':depid,'Home Goal':evgoal,'Away Goal':depgoal,'League':league,'Text':text,

    }
        datas = pd.DataFrame()
        for i in [1,2,3,4,5,10,15]:
            datas = pd.concat([datas,tables(url,i)],axis=1)
        return pd.concat([pd.DataFrame(data,index=[0]),datas],axis=1)
    except:
        return pd.DataFrame()
    
def hreftopla(url):
    hrefs = []
    start = 'http://soccer365.me'
    r= requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    for a in soup.find_all('a',class_='game_link',href=True):
        hrefs.append(start +a['href'])
    return hrefs

def manip(alldata):
    alldata = data.copy()
    alldata = alldata.replace('?',np.nan)
    alldata = alldata.replace('',np.nan)
    cols = [c for c in alldata.columns if 'offsides' not in c.lower()]
    alldata = alldata[cols]
    cols = [c for c in alldata.columns if 'fouls' not in c.lower()]
    alldata = alldata[cols]
    alldata = alldata.drop(['Home Goal','Away Goal'],axis=1)
    alldata = alldata.dropna()
    cols = [c for c in alldata.columns if 'rest' in c.lower()][2:]
    alldata = alldata.drop(cols,axis=1)
    alldata = alldata[alldata['League'].str.isnumeric()]
    alldata[['Home ID','Away ID']] = alldata[['Home ID','Away ID']].astype('int32')
    alldata['Round'] = alldata['Text'].str.findall(r'[0-9]+-round')
    alldata['Round'] = [','.join(map(str, l)) for l in alldata['Round']]
    alldata['Round'] = alldata['Round'].str.split('-',expand=True)[0].replace('',0).astype('int32')
    alldata['Text'] = alldata['Text'].str.findall(r'(\d{2}.\d{2}.\d{4}) (\d{2}:\d{2})')
    alldata['Text'] = [','.join(map(str, l)) for l in alldata['Text']]
    alldata[['Date','Time']] = alldata['Text'].str.replace("'",'').str.strip('()').str.split(',',expand=True)
    alldata[['Day','Month','Year']] = alldata['Date'].str.split('.',expand=True)
    alldata[['Hour','Minute']] = alldata['Time'].str.split(':',expand=True)
    alldata = alldata.drop(['Text','Date','Time'],axis=1)
    alldata = alldata.rename(columns={'1_Home Rest: Days (Hours)':'Home Rest','1_Away Rest: Days (Hours)':'Away Rest'})
    alldata['Home Rest'] = alldata['Home Rest'].str.split(' ',expand=True)[1]
    alldata['Away Rest'] = alldata['Away Rest'].str.split(' ',expand=True)[0]
    columns = ['Home Wins','Home Draws','Home Losing','Home Clean sheets','Home both teams to score','Home TU 2.5','Home TO 2.5']
    cols = [[c for c in alldata.columns if a in c]for a in columns]
    cols = [item for sublist in cols for item in sublist]
    for col in alldata[cols].columns:
        alldata[col] = alldata[col].str.split('%',expand=True)[0].astype('int32')/100
    columns = ['Away Wins','Away Draws','Away Losing','Away Clean sheets','Away both teams to score','Away TU 2.5','Away TO 2.5']
    cols = [[c for c in alldata.columns if a in c]for a in columns]
    cols = [item for sublist in cols for item in sublist]
    for col in alldata[cols].columns:
        alldata[col] = alldata[col].str.split('%',expand=True)[0].str.split(' ',expand=True)[1].fillna(0).astype('int32')/100
    alldata[alldata.select_dtypes(include=['object']).columns[3:-5]] = alldata[alldata.select_dtypes(include=['object']).columns[3:-5]].astype('float64')
    alldata['League'] = alldata['League'].astype('int32')
    stdf = alldata.copy()
    colsforst = stdf.columns[31:55]
    for i in range(2,6):
        if i == 2:
            stdf[alldata[colsforst.str.replace('1',f'{i}')].add_suffix('_OM').columns]=(alldata[colsforst.str.replace('1',f'{i}')].values *i ) - alldata[colsforst].values
        else:
            stdf[alldata[colsforst.str.replace('1',f'{i}')].add_suffix('_OM').columns]=(alldata[colsforst.str.replace('1',f'{i}')].values *i ) - (alldata[colsforst.str.replace('1',f'{i-1}')].values *(i-1) )
    deneme = stdf.copy()
    for col in colsforst:
        deneme[f"{col.replace('1','2')}_STD"] = stdf[[col,f"{col.replace('1','2')}_OM"]].std(axis=1)
        deneme[f"{col.replace('1','3')}_STD"] = stdf[[col,f"{col.replace('1','2')}_OM",f"{col.replace('1','3')}_OM"]].std(axis=1)
        deneme[f"{col.replace('1','4')}_STD"] = stdf[[col,f"{col.replace('1','2')}_OM",f"{col.replace('1','3')}_OM",f"{col.replace('1','4')}_OM"]].std(axis=1)
        deneme[f"{col.replace('1','5')}_STD"] = stdf[[col,f"{col.replace('1','2')}_OM",f"{col.replace('1','3')}_OM",f"{col.replace('1','4')}_OM",f"{col.replace('1','5')}_OM"]].std(axis=1)
    deneme = deneme.dropna()
    deneme[['Day','Month','Year','Hour','Minute']] = deneme.select_dtypes('object').iloc[:,2:].astype('int32')
    deneme = deneme.drop(['1_Home Goals scored','1_Home Goals conceded','1_Away Goals conceded','1_Away Goals scored'],axis=1)
    df = deneme.copy()
    cols = df.columns[5:27].drop(['Home Rest','Away Rest'])
    ddff = df.copy()
    for i in range(2,6):
        if i ==2:
            ddff[df[cols.str.replace('1',f'{i}')].add_suffix('_OM').columns]=(df[cols.str.replace('1',f'{i}')].values *i ) - df[cols].values
        else:
            ddff[df[cols.str.replace('1',f'{i}')].add_suffix('_OM').columns]=(df[cols.str.replace('1',f'{i}')].values *i ) - (df[cols.str.replace('1',f'{i-1}')].values *(i-1) )
    deneme = ddff.copy()
    for col in cols:
        deneme[f"{col.replace('1','2')}_STD"] = ddff[[col,f"{col.replace('1','2')}_OM"]].std(axis=1)
        deneme[f"{col.replace('1','3')}_STD"] = ddff[[col,f"{col.replace('1','2')}_OM",f"{col.replace('1','3')}_OM"]].std(axis=1)
        deneme[f"{col.replace('1','4')}_STD"] = ddff[[col,f"{col.replace('1','2')}_OM",f"{col.replace('1','3')}_OM",f"{col.replace('1','4')}_OM"]].std(axis=1)
        deneme[f"{col.replace('1','5')}_STD"] = ddff[[col,f"{col.replace('1','2')}_OM",f"{col.replace('1','3')}_OM",f"{col.replace('1','4')}_OM",f"{col.replace('1','5')}_OM"]].std(axis=1)
    return deneme

def tahminet(df):
    alldata = df.copy()
    df[['Home ID','Away ID','Day','Month','Year','Hour','Minute','League','Round']] = df[['Home ID','Away ID','Day','Month','Year','Hour','Minute','League','Round']].astype('category')
    pr=df.copy()
    df = df.drop(['Home Team','Away Team'],axis=1)
    msmodel = pickle.load(open('msmodel.sav','rb'))
    ustmodel = pickle.load(open('ustmodel.sav','rb'))
    bustmodel = pickle.load(open('bustmodel.sav','rb'))
    uustmodel = pickle.load(open('uustmodel.sav','rb'))
    kgmodel = pickle.load(open('kgmodel.sav','rb'))
    x1model = pickle.load(open('x1model.sav','rb'))
    x2model = pickle.load(open('x2model.sav','rb'))
    bustx1model = pickle.load(open('bustx1model.sav','rb'))
    bustx2model = pickle.load(open('bustx2model.sav','rb'))
    uustx1model = pickle.load(open('uustx1model.sav','rb'))
    uustx2model = pickle.load(open('uustx2model.sav','rb'))
    hsustmodel = pickle.load(open('hsust.sav','rb'))
    hbustmodel = pickle.load(open('hbust.sav','rb'))
    hustmodel = pickle.load(open('hust.sav','rb'))
    asustmodel = pickle.load(open('asust.sav','rb'))
    abustmodel = pickle.load(open('abust.sav','rb'))
    austmodel = pickle.load(open('aust.sav','rb'))
    evh1model = pickle.load(open('evh1model.sav','rb'))
    
    pr['X1'] = x1model.predict(df)
    pr['X2'] = x2model.predict(df)
    pr['BUSTX1'] = bustx1model.predict(df)
    pr['BUSTX2'] = bustx2model.predict(df)
    pr['UUSTX1'] = uustx1model.predict(df)
    pr['UUSTX2'] = uustx2model.predict(df)
    pr['HSUST'] = hsustmodel.predict(df)
    pr['HBUST'] = hbustmodel.predict(df)
    pr['HUST'] = hustmodel.predict(df)
    pr['ASUST'] = asustmodel.predict(df)
    pr['ABUST'] = abustmodel.predict(df)
    pr['AUST'] = austmodel.predict(df)
    pr['MS'] = msmodel.predict(df)
    pr['KG'] = kgmodel.predict(df)
    pr['BUST'] = bustmodel.predict(df)
    pr['UST'] = ustmodel.predict(df)
    pr['UUST'] = uustmodel.predict(df)
    pr['EVH1'] = evh1model.predict(df)
    pr['kg_prob'] = np.select([pr.KG==0,pr.KG==1], [kgmodel.predict_proba(df)[:,0],kgmodel.predict_proba(df)[:,1]], default=pr.KG)
    pr['ms1_prob'] = msmodel.predict_proba(df)[:,1]
    pr['ms0_prob'] = msmodel.predict_proba(df)[:,0]
    pr['ms2_prob'] = msmodel.predict_proba(df)[:,2]
    pr['bust_prob'] = np.select([pr.BUST==0,pr.BUST==1], [bustmodel.predict_proba(df)[:,0],bustmodel.predict_proba(df)[:,1]], default=pr.BUST)
    pr['ust_prob'] = np.select([pr.UST==0,pr.UST==1], [ustmodel.predict_proba(df)[:,0],ustmodel.predict_proba(df)[:,1]], default=pr.UST)
    pr['uust_prob'] = np.select([pr.UUST==0,pr.UUST==1], [uustmodel.predict_proba(df)[:,0],uustmodel.predict_proba(df)[:,1]], default=pr.UUST)
    pr['x1_prob'] = np.select([pr['X1']==0,pr['X1']==1], [x1model.predict_proba(df)[:,0],x1model.predict_proba(df)[:,1]], default=pr['X1'])
    pr['x2_prob'] = np.select([pr['X2']==0,pr['X2']==1], [x2model.predict_proba(df)[:,0],x2model.predict_proba(df)[:,1]], default=pr['X2'])
    pr['bustx1_prob'] = bustx1model.predict_proba(df)[:,1]
    pr['bustx2_prob'] = bustx2model.predict_proba(df)[:,1]
    pr['uustx1_prob'] = uustx1model.predict_proba(df)[:,1]
    pr['uustx2_prob'] = uustx2model.predict_proba(df)[:,1]
    pr['hsust_prob'] = np.select([pr['HSUST']==0,pr['HSUST']==1], [hsustmodel.predict_proba(df)[:,0],hsustmodel.predict_proba(df)[:,1]], default=pr['HSUST'])
    pr['hbust_prob'] = np.select([pr['HBUST']==0,pr['HBUST']==1], [hbustmodel.predict_proba(df)[:,0],hbustmodel.predict_proba(df)[:,1]], default=pr['HBUST'])
    pr['hust_prob'] = np.select([pr['HUST']==0,pr['HUST']==1], [hustmodel.predict_proba(df)[:,0],hustmodel.predict_proba(df)[:,1]], default=pr['HUST'])
    pr['asust_prob'] = np.select([pr['ASUST']==0,pr['ASUST']==1], [asustmodel.predict_proba(df)[:,0],asustmodel.predict_proba(df)[:,1]], default=pr['ASUST'])
    pr['abust_prob'] = np.select([pr['ABUST']==0,pr['ABUST']==1], [abustmodel.predict_proba(df)[:,0],abustmodel.predict_proba(df)[:,1]], default=pr['ABUST'])
    pr['aust_prob'] = np.select([pr['AUST']==0,pr['AUST']==1], [austmodel.predict_proba(df)[:,0],austmodel.predict_proba(df)[:,1]], default=pr['AUST'])
    pr['evh1_prob'] = np.select([pr['EVH1']==0,pr['EVH1']==1], [evh1model.predict_proba(df)[:,0],evh1model.predict_proba(df)[:,1]], default=pr['EVH1'])
    pr[['Home Team','Away Team']] = alldata[['Home Team','Away Team']]
    lastdf = pd.concat([pr[['Home Team','Away Team']],pr.iloc[:,46:]],axis=1)
    return lastdf

In [3]:
url = "https://soccer365.me/online/&tab=3"
options = Options()
#options.add_argument('headless')
webD = wb.Chrome('chromedriver.exe',chrome_options=options)
webD.create_options()
webD.get(url)

In [4]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [5]:
elements = webD.find_elements_by_xpath('//*[@class="game_link"]')
links=[]
for element in elements:
    links.append(element.get_attribute('href'))
links

['https://soccer365.me/live/match-konyaspor-trabzonspor-smotret-onlayn/',
 'https://soccer365.me/live/match-giresunspor-fenerbahche-smotret-onlayn/',
 'https://soccer365.me/live/match-alanyaspor-kasyimpasha-smotret-onlayn/',
 'https://soccer365.me/live/match-granada-real-sosedad-smotret-onlayn/',
 'https://soccer365.me/live/match-osasuna-betis-smotret-onlayn/',
 'https://soccer365.me/live/match-kadis-barselona-smotret-onlayn/',
 'https://soccer365.me/live/match-sampdoriya-napoli-smotret-onlayn/',
 'https://soccer365.me/live/match-torino-latsio-smotret-onlayn/',
 'https://soccer365.me/live/match-udineze-roma-smotret-onlayn/',
 'https://soccer365.me/live/match-zenitijevsk-tsska-m-smotret-onlayn/',
 'https://soccer365.me/live/match-torpedo-vladimir-ural-smotret-onlayn/',
 'https://soccer365.me/live/match-gent-anderleht-smotret-onlayn/',
 'https://soccer365.me/live/match-kambyur-herakles-smotret-onlayn/',
 'https://soccer365.me/live/match-az-tvente-smotret-onlayn/',
 'https://soccer365.me/

In [6]:
len(links)

147

In [7]:
data = pd.concat(pqdm(links,yarat,n_jobs=24),ignore_index=True)

SUBMITTING | :   0%|          | 0/147 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/147 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/147 [00:00<?, ?it/s]

In [8]:
df = tahminet(manip(data))

In [9]:
ms1ler = df.sort_values(['ms1_prob'],ascending=False).head(60)[['Home Team','Away Team','MS','ms1_prob','BUST','UST','UUST','bust_prob','ust_prob','uust_prob']]
ms0lar = df.sort_values(['ms0_prob'],ascending=False).head(60)[['Home Team','Away Team','MS','ms0_prob','BUST','UST','UUST','bust_prob','ust_prob','uust_prob']]
ms2ler = df.sort_values(['ms2_prob'],ascending=False).head(60)[['Home Team','Away Team','MS','ms2_prob','BUST','UST','UUST','bust_prob','ust_prob','uust_prob']]
evh1 = df.sort_values(['evh1_prob'],ascending=False).head(60)[['Home Team','Away Team','EVH1','evh1_prob','BUST','UST','UUST','bust_prob','ust_prob','uust_prob']]
ustler = df[df['UST']==1].sort_values(['ust_prob'],ascending=False).head(60)[['Home Team','Away Team','UST','ust_prob','X1','X2','x1_prob','x2_prob']]
altlar = df[df['UST']==0].sort_values(['ust_prob'],ascending=False).head(60)[['Home Team','Away Team','UST','ust_prob','X1','X2','x1_prob','x2_prob']]
kgvar = df[df['KG']==1].sort_values(['kg_prob'],ascending=False).head(60)[['Home Team','Away Team','KG','kg_prob','X1','X2','x1_prob','x2_prob']]
kgyok = df[df['KG']==0].sort_values(['kg_prob'],ascending=False).head(60)[['Home Team','Away Team','KG','kg_prob','X1','X2','x1_prob','x2_prob']]
x1ler = df.sort_values(['x1_prob'],ascending=False).head(60)[['Home Team','Away Team','X1','x1_prob','BUST','UST','UUST','bust_prob','ust_prob','uust_prob']]
x2ler = df.sort_values(['x2_prob'],ascending=False).head(60)[['Home Team','Away Team','X2','x2_prob','BUST','UST','UUST','bust_prob','ust_prob','uust_prob']]
bustlerust = df[df['BUST']==1].sort_values(['bust_prob'],ascending=False).head(60)[['Home Team','Away Team','BUST','bust_prob','X1','X2','x1_prob','x2_prob']]
bustleralt = df[df['BUST']==0].sort_values(['bust_prob'],ascending=False).head(60)[['Home Team','Away Team','BUST','bust_prob','X1','X2','x1_prob','x2_prob']]
uustlerust = df[df['UUST']==1].sort_values(['uust_prob'],ascending=False).head(60)[['Home Team','Away Team','UUST','uust_prob','X1','X2','x1_prob','x2_prob']]
uustleralt = df[df['UUST']==0].sort_values(['uust_prob'],ascending=False).head(60)[['Home Team','Away Team','UUST','uust_prob','X1','X2','x1_prob','x2_prob']]
bustx1ler = df.sort_values(['bustx1_prob'],ascending=False).head(60)[['Home Team','Away Team','BUSTX1','bustx1_prob']]
bustx2ler = df.sort_values(['bustx2_prob'],ascending=False).head(60)[['Home Team','Away Team','BUSTX2','bustx2_prob']]
uustx1ler = df.sort_values(['uustx1_prob'],ascending=False).head(60)[['Home Team','Away Team','UUSTX1','uustx1_prob']]
uustx2ler = df.sort_values(['uustx2_prob'],ascending=False).head(60)[['Home Team','Away Team','UUSTX2','uustx2_prob']]
hsustlerust = df[df['HSUST']==1].sort_values(['hsust_prob'],ascending=False).head(60)[['Home Team','Away Team','HSUST','hsust_prob']]
hsustleralt = df[df['HSUST']==0].sort_values(['hsust_prob'],ascending=False).head(60)[['Home Team','Away Team','HSUST','hsust_prob']]
hbustlerust = df[df['HBUST']==1].sort_values(['hbust_prob'],ascending=False).head(60)[['Home Team','Away Team','HBUST','hbust_prob']]
hbustleralt = df[df['HBUST']==0].sort_values(['hbust_prob'],ascending=False).head(60)[['Home Team','Away Team','HBUST','hbust_prob']]
hustlerust = df[df['HUST']==1].sort_values(['hust_prob'],ascending=False).head(60)[['Home Team','Away Team','HUST','hust_prob']]
hustleralt = df[df['HUST']==0].sort_values(['hust_prob'],ascending=False).head(60)[['Home Team','Away Team','HUST','hust_prob']]
asustlerust = df[df['ASUST']==1].sort_values(['asust_prob'],ascending=False).head(60)[['Home Team','Away Team','ASUST','asust_prob']]
asustleralt = df[df['ASUST']==0].sort_values(['asust_prob'],ascending=False).head(60)[['Home Team','Away Team','ASUST','asust_prob']]
abustlerust = df[df['ABUST']==1].sort_values(['abust_prob'],ascending=False).head(60)[['Home Team','Away Team','ABUST','abust_prob']]
abustleralt = df[df['ABUST']==0].sort_values(['abust_prob'],ascending=False).head(60)[['Home Team','Away Team','ABUST','abust_prob']]
austlerust = df[df['AUST']==1].sort_values(['aust_prob'],ascending=False).head(60)[['Home Team','Away Team','AUST','aust_prob']]
austleralt = df[df['AUST']==0].sort_values(['aust_prob'],ascending=False).head(60)[['Home Team','Away Team','AUST','aust_prob']]
ms2ters = df[df['X1']==0].sort_values(['x1_prob'],ascending=False).head(60)[['Home Team','Away Team','X1','x1_prob']]
ms1ters = df[df['X2']==0].sort_values(['x2_prob'],ascending=False).head(60)[['Home Team','Away Team','X2','x2_prob']]

In [10]:
len(df)

69

In [368]:
hbustlerust

,Home Team,Away Team,HBUST,hbust_prob
79,Celtic,Ross County,1,0.952119
66,Sachter Soligorsk,Sputnik Rechitsa,1,0.942213
278,Breidablik,Valur,1,0.913198
189,Nagoya Grampus,Tokushima Vortis,1,0.871140
59,Eyupspor,Denizlispor,1,0.867646
101,Hamburger,Sandhausen,1,0.803564
183,Cerezo Osaka,Consadole Sapporo,1,0.788041
182,Lillestrøm,Brann,1,0.782797
113,Young Boys,Zurich,1,0.775052
9,Man Utd,Newcastle Utd,1,0.766226


In [377]:
data[data['Home Team']=='Besiktas']

,Home Team,Home ID,Away Team,Away ID,Home Goal,Away Goal,League,Text,1_Home Games,1_Home Rest: Days (Hours),1_Home Wins,1_Home Draws,1_Home Losing,1_Home Goals scored,1_Home Goals conceded,1_Home Goals scored per game,1_Home Goals conceded per game,1_Home Clean sheets,1_Home both teams to score,1_Home TU 2.5,1_Home TO 2.5,1_Away Games,1_Away Rest: Days (Hours),1_Away Wins,1_Away Draws,1_Away Losing,1_Away Goals scored,1_Away Goals conceded,1_Away Goals scored per game,1_Away Goals conceded per game,1_Away Clean sheets,1_Away both teams to score,1_Away TU 2.5,1_Away TO 2.5,1_Home Shots,1_Home Shots (competitor),1_Home Shots on target,1_Home Shots on target (competitor),1_Home Possession,1_Home Possession (competitor),1_Home Corner,1_Home Corner (competitor),1_Home Fouls,1_Home Fouls (competitor),1_Home Offsides,1_Home Offsides (competitor),1_Home Yellow cards,1_Home Yellow cards (competitor),1_Home Red cards,1_Home Red cards (competitor),1_Away Shots,1_Away Shots (competitor),1_Away Shots on target,1_Away Shots on target (competitor),1_Away Possession,1_Away Possession (competitor),1_Away Corner,1_Away Corner (competitor),1_Away Fouls,1_Away Fouls (competitor),1_Away Offsides,1_Away Offsides (competitor),1_Away Yellow cards,1_Away Yellow cards (competitor),1_Away Red cards,1_Away Red cards (competitor),2_Home Games,2_Home Rest: Days (Hours),2_Home Wins,2_Home Draws,2_Home Losing,2_Home Goals scored,2_Home Goals conceded,2_Home Goals scored per game,2_Home Goals conceded per game,2_Home Clean sheets,2_Home both teams to score,2_Home TU 2.5,2_Home TO 2.5,2_Away Games,2_Away Rest: Days (Hours),2_Away Wins,2_Away Draws,2_Away Losing,2_Away Goals scored,2_Away Goals conceded,2_Away Goals scored per game,2_Away Goals conceded per game,2_Away Clean sheets,2_Away both teams to score,2_Away TU 2.5,2_Away TO 2.5,2_Home Shots,2_Home Shots (competitor),2_Home Shots on target,2_Home Shots on target (competitor),2_Home Possession,2_Home Possession (competitor),2_Home Corner,2_Home Corner (competitor),2_Home Fouls,2_Home Fouls (competitor),2_Home Offsides,2_Home Offsides (competitor),2_Home Yellow cards,2_Home Yellow cards (competitor),2_Home Red cards,2_Home Red cards (competitor),2_Away Shots,2_Away Shots (competitor),2_Away Shots on target,2_Away Shots on target (competitor),2_Away Possession,2_Away Possession (competitor),2_Away Corner,2_Away Corner (competitor),2_Away Fouls,2_Away Fouls (competitor),2_Away Offsides,2_Away Offsides (competitor),2_Away Yellow cards,2_Away Yellow cards (competitor),2_Away Red cards,2_Away Red cards (competitor),3_Home Games,3_Home Rest: Days (Hours),3_Home Wins,3_Home Draws,3_Home Losing,3_Home Goals scored,3_Home Goals conceded,3_Home Goals scored per game,3_Home Goals conceded per game,3_Home Clean sheets,3_Home both teams to score,3_Home TU 2.5,3_Home TO 2.5,3_Away Games,3_Away Rest: Days (Hours),3_Away Wins,3_Away Draws,3_Away Losing,3_Away Goals scored,3_Away Goals conceded,3_Away Goals scored per game,3_Away Goals conceded per game,3_Away Clean sheets,3_Away both teams to score,3_Away TU 2.5,3_Away TO 2.5,3_Home Shots,3_Home Shots (competitor),3_Home Shots on target,3_Home Shots on target (competitor),3_Home Possession,3_Home Possession (competitor),3_Home Corner,3_Home Corner (competitor),3_Home Fouls,3_Home Fouls (competitor),3_Home Offsides,3_Home Offsides (competitor),3_Home Yellow cards,3_Home Yellow cards (competitor),3_Home Red cards,3_Home Red cards (competitor),3_Away Shots,3_Away Shots (competitor),3_Away Shots on target,3_Away Shots on target (competitor),3_Away Possession,3_Away Possession (competitor),3_Away Corner,3_Away Corner (competitor),3_Away Fouls,3_Away Fouls (competitor),3_Away Offsides,3_Away Offsides (competitor),3_Away Yellow cards,3_Away Yellow cards (competitor),3_Away Red cards,3_Away Red cards (competitor),4_Home Games,4_Home Rest: Days (Hours),4_Home Wins,4_Home Draws,4_Home Losing,4_Home Goals scored,4_Home Goals conceded,4_Home Goals scored per game,4_Home Goals concede

In [379]:
df[df['Home Team']=='Besiktas']

,Home Team,Away Team,1_Away Corner (competitor),1_Away Yellow cards,1_Away Yellow cards (competitor),1_Away Red cards,1_Away Red cards (competitor),2_Home Games,2_Home Wins,2_Home Draws,2_Home Losing,2_Home Goals scored,2_Home Goals conceded,2_Home Goals scored per game,2_Home Goals conceded per game,2_Home Clean sheets,2_Home both teams to score,2_Home TU 2.5,2_Home TO 2.5,2_Away Games,2_Away Wins,2_Away Draws,2_Away Losing,2_Away Goals scored,2_Away Goals conceded,2_Away Goals scored per game,2_Away Goals conceded per game,2_Away Clean sheets,2_Away both teams to score,2_Away TU 2.5,2_Away TO 2.5,2_Home Shots,2_Home Shots (competitor),2_Home Shots on target,2_Home Shots on target (competitor),2_Home Possession,2_Home Possession (competitor),2_Home Corner,2_Home Corner (competitor),2_Home Yellow cards,2_Home Yellow cards (competitor),2_Home Red cards,2_Home Red cards (competitor),2_Away Shots,2_Away Shots (competitor),2_Away Shots on target,2_Away Shots on target (competitor),2_Away Possession,2_Away Possession (competitor),2_Away Corner,2_Away Corner (competitor),2_Away Yellow cards,2_Away Yellow cards (competitor),2_Away Red cards,2_Away Red cards (competitor),3_Home Games,3_Home Wins,3_Home Draws,3_Home Losing,3_Home Goals scored,3_Home Goals conceded,3_Home Goals scored per game,3_Home Goals conceded per game,3_Home Clean sheets,3_Home both teams to score,3_Home TU 2.5,3_Home TO 2.5,3_Away Games,3_Away Wins,3_Away Draws,3_Away Losing,3_Away Goals scored,3_Away Goals conceded,3_Away Goals scored per game,3_Away Goals conceded per game,3_Away Clean sheets,3_Away both teams to score,3_Away TU 2.5,3_Away TO 2.5,3_Home Shots,3_Home Shots (competitor),3_Home Shots on target,3_Home Shots on target (competitor),3_Home Possession,3_Home Possession (competitor),3_Home Corner,3_Home Corner (competitor),3_Home Yellow cards,3_Home Yellow cards (competitor),3_Home Red cards,3_Home Red cards (competitor),3_Away Shots,3_Away Shots (competitor),3_Away Shots on target,3_Away Shots on target (competitor),3_Away Possession,3_Away Possession (competitor),3_Away Corner,3_Away Corner (competitor),3_Away Yellow cards,3_Away Yellow cards (competitor),3_Away Red cards,3_Away Red cards (competitor),4_Home Games,4_Home Wins,4_Home Draws,4_Home Losing,4_Home Goals scored,4_Home Goals conceded,4_Home Goals scored per game,4_Home Goals conceded per game,4_Home Clean sheets,4_Home both teams to score,4_Home TU 2.5,4_Home TO 2.5,4_Away Games,4_Away Wins,4_Away Draws,4_Away Losing,4_Away Goals scored,4_Away Goals conceded,4_Away Goals scored per game,4_Away Goals conceded per game,4_Away Clean sheets,4_Away both teams to score,4_Away TU 2.5,4_Away TO 2.5,4_Home Shots,4_Home Shots (competitor),4_Home Shots on target,4_Home Shots on target (competitor),4_Home Possession,4_Home Possession (competitor),4_Home Corner,4_Home Corner (competitor),4_Home Yellow cards,4_Home Yellow cards (competitor),4_Home Red cards,4_Home Red cards (competitor),4_Away Shots,4_Away Shots (competitor),4_Away Shots on target,4_Away Shots on target (competitor),4_Away Possession,4_Away Possession (competitor),4_Away Corner,4_Away Corner (competitor),4_Away Yellow cards,4_Away Yellow cards (competitor),4_Away Red cards,4_Away Red cards (competitor),5_Home Games,5_Home Wins,5_Home Draws,5_Home Losing,5_Home Goals scored,5_Home Goals conceded,5_Home Goals scored per game,5_Home Goals conceded per game,5_Home Clean sheets,5_Home both teams to score,5_Home TU 2.5,5_Home TO 2.5,5_Away Games,5_Away Wins,5_Away Draws,5_Away Losing,5_Away Goals scored,5_Away Goals conceded,5_Away Goals scored per game,5_Away Goals conceded per game,5_Away Clean sheets,5_Away both teams to score,5_Away TU 2.5,5_Away TO 2.5,5_Home Shots,5_Home Shots (competitor),5_Home Shots on target,5_Home Shots on target (competitor),5_Home Possession,5_Home Possession (competitor),5_Home Corner,5_Home Corner (competitor),5_Home Yellow cards,5_Home Yellow cards (competitor),5_Home Red cards,5_Home Red cards (competitor),5_Aw

In [375]:
yuksekoran(df)[yuksekoran(df)['home team'].str.contains('Besikt')]

,home team,away team,max_prob,max_col,max_vals


In [ ]:
abustleral

In [372]:
bustlerust.style.highlight_max(axis=1,subset=['x1_prob','x2_prob'])

,Home Team,Away Team,BUST,bust_prob,X1,X2,x1_prob,x2_prob
24,Lorient,Lille,1,0.991599,1,1,0.802007,0.914783
182,Lillestrøm,Brann,1,0.990537,1,0,0.959982,0.685617
276,Akureyri,Fylkir,1,0.985920,1,0,0.959868,0.777199
278,Breidablik,Valur,1,0.983006,1,0,0.969816,0.872956
181,Stabaek,Viking,1,0.980590,1,1,0.866265,0.886695
79,Celtic,Ross County,1,0.980139,1,0,0.978799,0.881877
141,Colorado Rapids,Los Angeles,1,0.979141,1,1,0.950772,0.720058
1,Zenit,Ahmat,1,0.973885,1,0,0.934476,0.804889
81,Dundee,Livingston,1,0.973014,1,0,0.932966,0.617542
233,Maccabi Haifa,Bnei Sakhnin,1,0.971897,1,0,0.972619,0.790753


In [202]:
bustx1ler

,Home Team,Away Team,BUSTX1,bustx1_prob
10,Norway,Gibraltar,1,0.946801
55,Netherlands U21,Moldova U21,1,0.896047
128,Sao Paulo RS,Lajeadense,1,0.894354
124,Ilves-Kissat,KaaPo,1,0.892728
104,Campbelltown City,Sturt Lions,1,0.887675
58,England U21,Kosovo U21,1,0.871566
47,Russia U21,Malta U21,1,0.867379
115,Fjolnir,Vestmannaeyjar,1,0.836108
49,Austria U21,Azerbaijan U21,1,0.833181
62,Universidad Chile,Deportes La Serena,1,0.810483


In [271]:
x2ler.style.highlight_max(axis=1,subset=['bust_prob','ust_prob','uust_prob'])

,Home Team,Away Team,X2,x2_prob,BUST,UST,UUST,bust_prob,ust_prob,uust_prob
151,Bnei Sakhnin,Maccabi Tel Aviv,1,0.962135,1,0,0,0.783989,0.716658,0.928677
163,Leon,América,1,0.952234,1,0,0,0.921535,0.831842,0.952589
92,Vejle,FC Copenhagen,1,0.947969,1,0,0,0.946961,0.769657,0.837189
138,Domzale,Mura,1,0.946059,1,0,0,0.836894,0.813865,0.933820
70,Santos,Flamengo,1,0.946048,1,0,0,0.853502,0.636134,0.873427
106,Tromso,Bodø / Glimt,1,0.945587,1,1,0,0.953846,0.511697,0.868630
21,Reims,Paris SG,1,0.945248,1,1,0,0.917198,0.515753,0.824837
227,Royal Excel Mouscron,Waasland-Beveren,1,0.944947,1,0,0,0.851583,0.510422,0.925855
96,Kolomna,Saransk,1,0.942651,1,1,0,0.964030,0.823025,0.761519
234,Posusje,Zrinjski,1,0.940319,1,0,0,0.864897,0.829177,0.888983
